In [10]:
### Importing Dependencies

In [11]:
# Importing dependecies:
import re
import sys
import string
import pandas as pd
import numpy as np

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Configuring Notebook environment:
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = (10.0, 7.5)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [12]:
df = pd.read_csv('data/strings/df_clean.csv', index_col=0)
df.head()

,title,ingredients,instructions,ingredients_vector,instructions_vector
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,grammie hamblets deviled crab,celery finely chopped green pepper finely chop...,toss ingredients lightly spoon buttered baking...,"['celery', 'finely', 'chopped', 'green', 'pepp...","['toss', 'ingredients', 'lightly', 'spoon', 'b..."
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,infineon raceway baked beans,skirt steak cut inch dicekosher salt fresh cra...,watch make recipe sprinkle steak salt pepper s...,"['skirt', 'steak', 'cut', 'inch', 'dicekosher'...","['watch', 'make', 'recipe', 'sprinkle', 'steak..."
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,southwestern black bean dip,cups dried black beans picked rinsed cups wate...,saucepan let beans soak enough cold water cove...,"['cups', 'dried', 'black', 'beans', 'picked', ...","['saucepan', 'let', 'beans', 'soak', 'enough',..."
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,sour cream noodle bake,ground chuckone tomato sauce saltfreshly groun...,watch make recipe preheat oven degrees f brown...,"['ground', 'chuckone', 'tomato', 'sauce', 'sal...","['watch', 'make', 'recipe', 'preheat', 'oven',..."
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,sushi renovation,rice brown mediumgrain cookedcup quinoacup swe...,special equipment sushi mat cook brown rice qu...,"['rice', 'brown', 'mediumgrain', 'cookedcup', ...","['special', 'equipment', 'sushi', 'mat', 'cook..."


### Analyzing Recipes

In [21]:
df['ingredients_vector'].value_counts()

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   1475
['chopped', 'onion', 'tomatoes', 'grreen', 'chillies']                                                                                                                                                        

In [16]:
df['ingredients'].value_counts()

chopped onion tomatoes grreen chillies                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  8
cups sugar light corn syrup hot water butter margarine cups pecans vanilla flavoring heaping baking soda                                                                                                                                                                                                                                                                                                                                                                                      

In [19]:
df['title'].value_counts()

guacamole                                                           33
tiramisu                                                            23
fried chicken                                                       21
caesar salad                                                        19
pecan pie                                                           19
                                                                    ..
prickly pear iced tea                                                1
chicken pinwheels with tellurides own rub and sw seasoning           1
turmeric sweet cornstuffed tomato with puree of brussels sprouts     1
red wine cabbage                                                     1
meringue swirls                                                      1
Name: title, Length: 52474, dtype: int64